In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import pprint
import json
import os

In [2]:
# Import the files per year
combined_contract_data = '../Resources/Annual_HSS_Data_Cleaned.csv'
df_combined_contract = pd.read_csv(combined_contract_data, low_memory=False)


In [3]:
# Visualize it so that the columns and values can be referenced when needed
df_combined_contract.head()

,contract_transaction_unique_key,contract_award_unique_key,award_id_piid,modification_number,parent_award_agency_name,federal_action_obligation,total_dollars_obligated,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,...,community_developed_corporation_owned_firm,federal_agency,foreign_owned_and_located,for_profit_organization,dot_certified_disadvantage,self_certified_small_disadvantaged_business,small_disadvantaged_business,c8a_program_participant,historically_underutilized_business_zone_hubzone_firm,sba_certified_8a_joint_venture
0,7523_-NONE-_75D30118C01909_P00001_-NONE-_0,CONT_AWD_75D30118C01909_7523_-NONE-_-NONE-,75D30118C01909,P00001,NaN,0.0,295822.88,2019,2018-09-28 00:00:00,2019-06-14 00:00:00,...,f,f,f,t,t,t,f,t,t,f
1,7523_-NONE-_75D30118C02052_P00001_-NONE-_0,CONT_AWD_75D30118C02052_7523_-NONE-_-NONE-,75D30118C02052,P00001,NaN,0.0,152355.17,2019,2018-09-28 00:00:00,2019-09-15 00:00:00,...,f,f,f,t,t,t,f,t,t,f
2,7523_-NONE-_HHSD200201692404C_3_-NONE-_0,CONT_AWD_HHSD200201692404C_7523_-NONE-_-NONE-,HHSD200201692404C,3,NaN,0.0,1308998.24,2019,2018-10-01 00:00:00,2021-09-26 00:00:00,...,f,f,f,t,f,t,f,t,t,f
3,7523_-NONE-_HHSD200201692405C_3_-NONE-_0,CONT_AWD_HHSD200201692405C_7523_-NONE-_-NONE-,HHSD200201692405C,3,NaN,0.0,9700.00,2019,2018-10-01 00:00:00,2021-09-26 00:00:00,...,f,f,f,t,f,f,f,f,f,f
4,7523_-NONE-_HHSD200201692406C_8_-NONE-_0,CONT_AWD_HHSD200201692406C_7523_-NONE-_-NONE-,HHSD200201692406C,8,NaN,0.0,614109.66,2019,2018-10-01 00:00:00,2021-09-26 00:00:00,...,f,f,f,t,f,f,f,f,f,f


In [4]:
# Count the information in the columns
df_combined_contract.count()

contract_transaction_unique_key                               725993
contract_award_unique_key                                     725993
award_id_piid                                                 725993
modification_number                                           725993
parent_award_agency_name                                      435174
federal_action_obligation                                     725993
total_dollars_obligated                                       218979
action_date_fiscal_year                                       725993
period_of_performance_start_date                              725993
period_of_performance_current_end_date                        725992
awarding_agency_name                                          725993
awarding_sub_agency_name                                      725993
awarding_office_name                                          725992
funding_agency_name                                           725993
funding_sub_agency_name           

In [5]:
# Keep only the desired columns for a new dataframe to study the industry/service categories
# I chose federal action obligations to isolate the flow of funds each year, 
# The descriptive column (NAICS) in order to capture industry info, 
# The women owned and minority owned to capture demographic information, 
# And forgeign owned and located will allow differentiation between US/Foreign contract awards and that column was not missing any data
industry_df = df_combined_contract[["federal_action_obligation","action_date_fiscal_year",
"naics_description","foreign_owned_and_located","woman_owned_business","minority_owned_business"]]

In [6]:
# Viewing the new dataframe
industry_df.head(20)

,federal_action_obligation,action_date_fiscal_year,naics_description,foreign_owned_and_located,woman_owned_business,minority_owned_business
0,0.00,2019,CUSTOM COMPUTER PROGRAMMING SERVICES,f,f,t
1,0.00,2019,"ALL OTHER PROFESSIONAL, SCIENTIFIC, AND TECHNI...",f,f,f
2,0.00,2019,PHARMACEUTICAL PREPARATION MANUFACTURING,f,t,t
3,0.00,2019,PHARMACEUTICAL PREPARATION MANUFACTURING,f,f,f
4,0.00,2019,PHARMACEUTICAL PREPARATION MANUFACTURING,f,t,t
5,0.00,2019,PHARMACEUTICAL PREPARATION MANUFACTURING,f,f,f
6,0.00,2019,PHARMACEUTICAL PREPARATION MANUFACTURING,f,f,f
7,0.00,2019,PHARMACEUTICAL PREPARATION MANUFACTURING,f,f,t
8,0.00,2019,PHARMACEUTICAL PREPARATION MANUFACTURING,f,f,f
9,-5.64,2019,COMPUTER SYSTEMS DESIGN SERVICES,f,f,t


In [7]:
# Renaming the columns to make more sense
industry_df = industry_df.rename(columns={"federal_action_obligation": "Dollar Amount Obligated/De-Obligated","action_date_fiscal_year": "Fiscal Year",
"naics_description": "North American Industry Classification","foreign_owned_and_located":"US or Foreign Business",
                                          "woman_owned_business": "Women Owned Business","minority_owned_business": "Minority Owned Business"})

In [8]:
# Viewing the new dataframe
industry_df.head()

,Dollar Amount Obligated/De-Obligated,Fiscal Year,North American Industry Classification,US or Foreign Business,Women Owned Business,Minority Owned Business
0,0.0,2019,CUSTOM COMPUTER PROGRAMMING SERVICES,f,f,t
1,0.0,2019,"ALL OTHER PROFESSIONAL, SCIENTIFIC, AND TECHNI...",f,f,f
2,0.0,2019,PHARMACEUTICAL PREPARATION MANUFACTURING,f,t,t
3,0.0,2019,PHARMACEUTICAL PREPARATION MANUFACTURING,f,f,f
4,0.0,2019,PHARMACEUTICAL PREPARATION MANUFACTURING,f,t,t


In [9]:
# Checking to see what the datatypes are for the dataframe column values
industry_df.dtypes

Dollar Amount Obligated/De-Obligated      float64
Fiscal Year                                 int64
North American Industry Classification     object
US or Foreign Business                     object
Women Owned Business                       object
Minority Owned Business                    object
dtype: object

In [10]:
# Delete rows that contain FY 2011
industry_df = industry_df.set_index("Fiscal Year")
industry_df = industry_df.drop(2011, axis=0)
industry_df.reset_index(level=0, inplace=True)
industry_df.tail()

,Fiscal Year,Dollar Amount Obligated/De-Obligated,North American Industry Classification,US or Foreign Business,Women Owned Business,Minority Owned Business
644451,2012,644387.92,ADMINISTRATIVE MANAGEMENT AND GENERAL MANAGEME...,f,t,f
644452,2012,0.00,ADMINISTRATIVE MANAGEMENT AND GENERAL MANAGEME...,f,t,f
644453,2012,24694.29,COMPUTER SYSTEMS DESIGN SERVICES,f,f,f
644454,2012,-40260.00,BIOLOGICAL PRODUCT (EXCEPT DIAGNOSTIC) MANUFAC...,f,f,f
644455,2012,0.00,OTHER COMPUTER RELATED SERVICES,f,f,f


In [11]:
# Export final dataframe to CSV
industry_df.to_csv("../Resources/Industry_ContractData_Cleaned.csv", index=False)